In [1]:
demand = [990,1980,3961,2971,1980]   #value changes later 
d=0  # d% shortage allowance
Y_b = [1.3086,1.3671,1.4183,1.4538,1.5122]           # Fabric yield (consumption rate) rate per garment of size 𝛽 #value changes later 
f= 1.95 # Fabric cost 

U = 1
l_max= 20  #yrds
e=0

#Input variables (setup cost)
c_s=500  #setup cost
c_e=20. #excess production cost

P_min, P_max= 4,40

In [2]:
import numpy as np
import math
import pandas as pd
from copy import deepcopy
rng = np.random.default_rng()
import random
import time
import plotly.express as px
import plotly.graph_objects as go
from itertools import product

In [3]:
def Update_Res(R,GG,PP):
    for s in range(len(GG)): #Updating Residual within the while loop
        R=R-np.dot(GG[s],PP[s])
    return R

In [4]:
def Length(g_i_j):
    l_i = e+ np.dot(g_i_j,Y_b)/U
    return l_i


In [5]:
def Shortage_allowance(Q,d=0.01):
    temp=np.dot((1-d),Q)
    return [round(i) for i in temp]
Q_b= Shortage_allowance(demand,d)

# Q_b

In [6]:
from Heuristics import H1,H3,H5,H3_MB2016,H1_T2020

## Objective Function

In [9]:
def ObjectiveFunction (chromosome):
    
    G_a_b = chromosome['G']
    P_a = chromosome['P']
    Alpha = len(P_a)             # number of Sections
    

    '''                         Fabric Cost                      '''
        
    # Total fabric length = L # Total Fabric Cost = C_F

    l_a=[Length(G_a_b[alpha]) for alpha in range(Alpha) ] #Length function
    L= np.dot(l_a,P_a)       #Multiply then Sum
    C_F = L*f
    #print('Total Fabric Cost = C_F: ',C_F,'Length:',L)
    
    
    '''                        setup Cost                        '''
    
    
    C_S = 0
    for α in range(Alpha):
        if l_a[α]>e:   # this makes sure that section has at least one garments 
            C_S += c_s       
    #print('Total setup Cost = C_S: ',C_S)

    '''                        excess production Cost                        '''
        
    
    C_E=0
    R=Update_Res(R=Q_b, GG=G_a_b, PP=P_a)
    for r in R:
        if r<0:
            C_E += (-r* c_e)
    #print('Total excess prod Cost = C_E: ',C_E) 
    
    return C_F+C_S+C_E

In [10]:
# ObjectiveFunction(Sol_1)

### Time spend

## Fitness Score

In [11]:
def Fitness(chromosome): 
  
    G_a_b= chromosome['G']
    P_a = chromosome['P']
    Beta= len(G_a_b[0])
    
    score= ObjectiveFunction(chromosome)
    #print('score:',score)
    fitness_score=score
    
                
    '''       Penalty for shortage production           '''
    R= Update_Res(R=demand,GG=G_a_b,PP=P_a)
    for beta in range(Beta):
        if R[beta]>0:
            s_penalty= R[beta]/sum(demand)
            fitness_score +=score*s_penalty/2 
    
    
    '''       double check if the solution is valid       '''
    res= Update_Res(R=Q_b,GG=G_a_b,PP=P_a)
    if max(res)>0:
        print('solution is unvalid in demand constraint')
        fitness_score +=10000   #this will eventualy make the solution extinct.

    for i in G_a_b:
        if Length(i)>l_max:
            print('breaking !! fitness !! solution is not valid in length constraint: ',i, Y_b, l_max,Length(i))
            fitness_score +=10000
    
    return fitness_score

# Fitness(Sol_1)    

## Function Initial Population Generation

In [12]:
def GeneratePopulation(pop_size):
    sol1=H3_MB2016(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
    sol2=H1_T2020(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
    P_of_S=[]
    for sol in (sol1,sol2):
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    
    while len(P_of_S)<pop_size:
        h=rng.integers(0,3)
        #print('h:',h)
        if h==0:
            sol= H1(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        elif h==1:
            sol=H3(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)

        else:
            sol=H5(Q=Q_b,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
        i=0
        for g in sol['G']:
            if Length(g)>l_max:
                i+=1
        if i==0:
            P_of_S.append(sol)
    return P_of_S
Pool_of_Sol= GeneratePopulation(50)
len(Pool_of_Sol)

50

In [13]:
def S_with_F(p_o_s):
    for i in range(len(p_o_s)): 
        if 'F' not in p_o_s[i]:
            p_o_s[i]['F']=Fitness(p_o_s[i])
    return p_o_s

## PSO

### Cleaning section with zeros

In [14]:
def CleanZeros (Solution):
    j=0
    while j < len(Solution['G']):
        if max(Solution['G'][j])==0:
            Solution['G'].pop(j)
            Solution['P'].pop(j)
            continue
        j+=1

    #This is to make sure 
    if len(Solution['G'])!=len(Solution['P']):
        raise ValueError('P and G lengths are not same')
    
    return Solution

In [15]:
# CleanZeros(Sol_1)

## Velocity Update (Jarboui et al. 2008)

Lets assume 1st sol as X, 2nd Sol as Pbest, and 3rd Sol as Gbest

#### Now we have to calculate Y

##### Initial Velocity generator

In [16]:
def initial_velocity(Range, Sol): #Range is a list
    a,b= Range
    m=len(Sol['G'])
    
    #generate a random uniform array  [-a,b] of the same size of the solutions 
    
    v=(b-a) * np.random.random_sample(m) +a  #http://bit.ly/3To2OWe
    v=v.tolist()
    
    return {'V':v}

In [17]:
def Get_Y(X,GBest,PBest): #(Jarboui et al., 2008, p. 302)
    y=[]
    lens=[len(i) for i in [X['G'],GBest['G'],PBest['G']]]
    min_len=min(lens)
    
    for i in range(min_len):
        if X['G'][i]==GBest['G'][i] and X['G'][i]==PBest['G'][i]:
            y.append(random.choice([-1,1]))
        elif X['G'][i]==GBest['G'][i]:
            y.append(1)
        elif X['G'][i]==PBest['G'][i]:
            y.append(-1)
        else:
            y.append(0)
        
    return {'Y':y}

### Now we have to calculate Velocity

In [18]:
def New_V(Y,V,c1=1,c2=1,w=.75): #Parameter setting: (Jarboui et al., 2008, p. 306)

    for i in range(len(Y['Y'])):
        y=Y['Y'][i]
        v=V['V'][i]
        V['V'][i]= w*v+ rng.random()*c1*(-1-y)+rng.random()*c2*(1-y)
        
    return V

### Now we need to calculate λ

In [19]:
def Get_λ(Y,V):
    λ=[]
    for i in range(len(Y['Y'])):
        λ.append(Y['Y'][i]+V['V'][i])
    return {'λ':λ}
# λ=Get_λ(Y,V)
# λ

### Update X with Eq-10 (Jarboui et al., 2008, p. 303)

In [20]:
def Perturbation(xg,xp,R,p_rate):
    
    if rng.random()<p_rate:
        p1,p2=sorted([xp,min(P_max,max(P_min,max(R)))])
        xp= rng.integers(p1,p2+1)
    for j in range(len(xg)): #small purtubration (like mutaion)
        if rng.random()<p_rate:
            xg[j]=0
            temp= min(math.ceil(R[j]/xp),math.floor((l_max-Length(xg))/(Y_b[j]/U)))
            temp= max(0,temp)
            #xg[j]=max(0,temp)
            xg[j]=rng.integers(0,temp+1)
        if R[j]<=0 and xg[j]>0:
            xg[j]=0
            
    return xg,xp

def Update_X(X,GBest,PBest,λ, ϕ=0.5, p_rate=.1):

    XG=[]
    XP=[]
    R= Q_b
    for i in range(len(λ['λ'])):
        if λ['λ'][i] > ϕ:
            #print('Gbest')
            
            xg=GBest['G'][i]
            xp=GBest['P'][i]
                           
        elif λ['λ'][i] < -ϕ:
            #print('Pbest')
            
            xg=PBest['G'][i]
            xp=PBest['P'][i]


        else:
            #print('X')
            xg,xp= Perturbation(xg=X['G'][i],xp=X['P'][i],R=R,p_rate=p_rate) #Perturbation function

        
        if max(xg)>0:    
            XG.append(xg)
            XP.append(xp)
        
        R= Update_Res(R=Q_b,GG=XG,PP=XP)
        if max(R)<=0:
            return {'G':XG,'P':XP}
  
    for i in range(len(λ['λ']), len(X['G'])):        
        xg,xp= Perturbation(xg=X['G'][i],xp=X['P'][i],R=R, p_rate=p_rate) #Perturbation function
        
        if max(xg)>0:
            XG.append(xg)
            XP.append(xp)
        R= Update_Res(R=Q_b,GG=XG,PP=XP)
        if max(R)<=0:
            return {'G':XG,'P':XP}

    h=H1(Q=R,Y=Y_b,Pm=[P_min,P_max],U=U,e=e,l_max=l_max)
        
    g= h['G']
    p = h['P']    
    #print()
    #print(g,p) 
    #print()
    XG=XG+g
    XP=XP+p

    return {'G':XG,'P':XP}
# newX= Update_X(X,Gbest,Pbest,newY)
# newX

In [21]:
def Update_dimension(XX,VV, in_vel_range=[-0.5,0.5]):
    mm= len(XX['G'])
    m= len(VV['V'])
    
    if mm <= m:
        return {'V':VV['V'][:m]}
    else:
        a,b= in_vel_range
        v=(b-a) * np.random.random_sample(mm-m) +a  #http://bit.ly/3To2OWe
        v=v.tolist()
        V=VV['V']+v
        return {'V':V}

In [22]:
def Get_Gbest(p_o_s):
    gbest=p_o_s[0]
    for i in range(len(p_o_s)):
        if p_o_s[i]['F']<gbest['F']:
            gbest= deepcopy(p_o_s[i])
    return gbest

In [23]:
# newX= Update_X(X,Gbest,Pbest,newY)
# newX

In [24]:
# Fitness(newX)

In [25]:
#Pool_of_Sol

# Main PSO

In [26]:
def PSO(swarmsize,iteration,ϕ=.7,c1=2,c2=2,w=1,p_rate=.2):
    
    P_of_S= GeneratePopulation(swarmsize)
    P_of_S= S_with_F(P_of_S)
    P_of_Pbest=P_of_S
    
    in_vel_range=[-ϕ,ϕ]
    P_of_Velocity= [initial_velocity(in_vel_range,P_of_S[i]) for i in range(len(P_of_S))]
    
    Gbest=Get_Gbest(P_of_S)
    sc=0
    bests=[Gbest['F']]
    for i in range(iteration):
        for j in range(len(P_of_S)):
            X=P_of_S[j]
            Pbest=P_of_Pbest[j]
            V= P_of_Velocity[j]
            Y= Get_Y(X=X,GBest=Gbest,PBest=Pbest)

            newV= New_V(Y=Y,V=V,c1=c1,c2=c2,w=w)
            
            λ= Get_λ(Y=Y,V=newV)

            newX= Update_X(X=X,GBest=Gbest,PBest=Pbest,λ=λ,ϕ=ϕ, p_rate=p_rate)
            newX['F']= Fitness(newX)

            P_of_S[j]=deepcopy(newX)
            
            newV= Update_dimension(XX=newX,VV= newV, in_vel_range=in_vel_range)
            P_of_Velocity[j]= deepcopy(newV)
            
            if newX['F'] < Pbest['F']:
                P_of_Pbest[j]= deepcopy(newX)
            if newX['F'] < Gbest['F']:
                Gbest=deepcopy(newX)
                sc=i
        #print(Gbest, Fitness(Gbest))
        bests.append(Gbest['F'])
    
    # xx=[i for i in range(len(bests))]
    # fig=px.line(x=xx,
    #             y=bests,
    #             title=f'swarmsize={swarmsize},iteration= {iteration},ϕ={ϕ},c1= {c1},c2={c2},w={w}, Gbest={bests[-1]}',
    #             labels=dict(x='iteration',y='fitness'))
    # fig.show()
    
    return CleanZeros(Gbest),sc
# PSO(swarmsize=150,iteration=250)

## Application to problems

In [30]:
path='/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/DataSets.xlsx'

df=pd.read_excel(path,sheet_name='Sheet4',index_col=[0])

df=df.T

df.columns = df.columns.str.strip()

col=list(df.columns)
col= sorted(set(col), key=col.index)


df

Style,S4,S4,S4,S4,S5,S5,S5,S5,S5,S7,...,B5,B5,B5,B7,B7,B7,B7,B7,B7,B7
Demand,80.00,95.000,58.000,28.0,25.00,95.000,145.000,130.0,25.000,8.00,...,12498.0,2256.000,569.000,6780.00,14526.000,16473.000,18767.0,112498.000,8569.000,6532.0
Consumption,1.42,1.441,1.462,1.5,1.42,1.441,1.462,1.5,1.523,1.42,...,1.5,1.523,1.634,1.42,1.441,1.462,1.5,1.523,1.634,1.7
Price/unit length,10.00,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,10.00,...,NaN,NaN,NaN,10.00,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
col

['S4', 'S5', 'S7', 'M4', 'M5', 'M7', 'B4', 'B5', 'B7']

In [32]:
def PSO_with_sc(swarmsize,iteration,ϕ=.7,c1=2,c2=2,w=1,p_rate=.2,sc=100):
    
    P_of_S= GeneratePopulation(swarmsize)
    P_of_S= S_with_F(P_of_S)
    P_of_Pbest=P_of_S
    
    in_vel_range=[-ϕ,ϕ]
    P_of_Velocity= [initial_velocity(in_vel_range,P_of_S[i]) for i in range(len(P_of_S))]
    
    Gbest=Get_Gbest(P_of_S)
    it=0
    bests=[Gbest['F']]
    for i in range(iteration):
        for j in range(len(P_of_S)):
            X=P_of_S[j]
            Pbest=P_of_Pbest[j]
            V= P_of_Velocity[j]
            Y= Get_Y(X=X,GBest=Gbest,PBest=Pbest)

            newV= New_V(Y=Y,V=V,c1=c1,c2=c2,w=w)
            
            λ= Get_λ(Y=Y,V=newV)

            newX= Update_X(X=X,GBest=Gbest,PBest=Pbest,λ=λ,ϕ=ϕ, p_rate=p_rate)
            newX['F']= Fitness(newX)

            P_of_S[j]=deepcopy(newX)
            
            newV= Update_dimension(XX=newX,VV= newV, in_vel_range=in_vel_range)
            P_of_Velocity[j]= deepcopy(newV)
            
            if newX['F'] < Pbest['F']:
                P_of_Pbest[j]= deepcopy(newX)
            if newX['F'] < Gbest['F']:
                Gbest=deepcopy(newX)

        #print(Gbest, Fitness(Gbest))
        bests.append(Gbest['F'])
        if bests[-1] < bests[-2]:
            it=0
        else:
            it+=1
        if it>sc:
            break
    
#     xx=[i for i in range(len(bests))]
#     fig=px.line(x=xx,
#                 y=bests,
#                 title=f'swarmsize={swarmsize},iteration= {iteration},ϕ={ϕ},c1= {c1},c2={c2},w={w}, Gbest={bests[-1]}',
#                 labels=dict(x='iteration',y='fitness'))
#     fig.show()
    
    return CleanZeros(Gbest)
# PSO(swarmsize=150,iteration=250)

## Selected Parameters : 
* c1=c2= 2, 
* ϕ= 0.5, 
* w= 0.4, 
* p_rate=0.2 
* sc=90 (stoping criteria)
* swarmsize= 250 
* iteration= 200

In [ ]:
df_result_PSO = pd.DataFrame(columns=['style','demand','sol','Fitness','Cost','Runtime','P_cost','P_time'])

c1,c2= 2,2 
ϕ= 0.5
w= 0.4 
p_rate=0.2 
sc=90 #stoping criteria
swarmsize= 250 
iteration= 200  #parameters
for j in range(10):
    for i in col:
        demand=df[i].loc['Demand'].to_list()
        demand=list(map(int,demand))
        #print('demand=',demand)

        d=0
        Q_b= Shortage_allowance(demand,d)

        consumption=df[i].loc['Consumption'].to_list()
        Y_b=list(map(float,consumption))
        #print('consumption=',Y_b)

        price=df[i].loc['Price/unit length'].to_list()
        price=list(map(float,price))
        f=price[0]

        print(i,demand,Q_b,Y_b,f)
        time_start = time.perf_counter()
        Gbest = PSO_with_sc(swarmsize=swarmsize,iteration=iteration,ϕ=ϕ,c1=c1,c2=c2,w=w,p_rate=p_rate,sc=sc)
        time_elapsed = (time.perf_counter() - time_start)
        print(Gbest, 'time:', time_elapsed)
        df_result_PSO=df_result_PSO.append({'style':i,'demand':demand,'Cost': ObjectiveFunction(Gbest),'sol':Gbest,
                                            'Fitness':Gbest['F'],'Runtime': time_elapsed},
                                           ignore_index=True)
    df_result_PSO.to_excel('/Users/sharif-al-mahmud/Library/Mobile Documents/com~apple~CloudDocs/Thesis/Data/PSO output for Tsao et al data.xlsx')

B7 [6780, 14526, 16473, 18767, 112498, 8569, 6532] [6780, 14526, 16473, 18767, 112498, 8569, 6532] [1.42, 1.441, 1.462, 1.5, 1.523, 1.634, 1.7] 10.0


In [ ]:
df_result_PSO